# TELECOM CHURN PREDICTION

In [1]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

In [2]:
# Loading My Dataset and Creating a DataFrame

df = pd.read_csv("cell2cell-duke univeristy.csv")
df.head()

,Unnamed: 0,X,customer,traintest,churn,churndep,revenue,mou,recchrge,directas,...,retaccpt,newcelly,newcelln,refer,incmiss,income,mcycle,setprcm,setprc,retcall
0,1,1,1000002,0,0,NaN,57.492500,482.75,37.424999,0.2475,...,0,0,1,0,0,5,0,0,149.989990,0
1,2,2,1000006,0,0,NaN,82.275002,1312.25,75.000000,1.2375,...,0,1,0,0,0,6,0,0,9.989998,0
2,3,3,1000010,0,0,NaN,31.662500,25.50,29.990000,0.2475,...,0,0,1,0,0,9,0,0,29.989990,0
3,4,4,1000011,0,0,NaN,62.127499,97.50,65.985001,2.4750,...,0,1,0,0,0,6,0,0,29.989990,0
4,5,5,1000014,0,0,NaN,25.225000,2.50,25.000000,0.0000,...,0,1,0,0,0,7,0,0,29.989990,0


In [3]:
df.shape

(71047, 72)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71047 entries, 0 to 71046
Data columns (total 72 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  71047 non-null  int64  
 1   X           71047 non-null  int64  
 2   customer    71047 non-null  int64  
 3   traintest   71047 non-null  int64  
 4   churn       71047 non-null  int64  
 5   churndep    40000 non-null  float64
 6   revenue     70831 non-null  float64
 7   mou         70831 non-null  float64
 8   recchrge    70831 non-null  float64
 9   directas    70831 non-null  float64
 10  overage     70831 non-null  float64
 11  roam        70831 non-null  float64
 12  changem     70545 non-null  float64
 13  changer     70545 non-null  float64
 14  dropvce     71047 non-null  float64
 15  blckvce     71047 non-null  float64
 16  unansvce    71047 non-null  float64
 17  custcare    71047 non-null  float64
 18  threeway    71047 non-null  float64
 19  mourec      71047 non-nul